# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo SINASC_RO_2019.csv` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo `estrutura_sinasc_para_CD.pdf` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
`
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis `['ESTCIVMAE', 'CONSULTAS']` possuem o código `9`, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código `9`.<br>
6. Substitua os valores faltantes da quantitativa (`QTDFILVIVO`) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção `inplace = True`.

In [64]:
import pandas as pd
import requests

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

In [66]:
sinasc.dtypes

ORIGEM          int64
CODESTAB      float64
CODMUNNASC      int64
LOCNASC         int64
IDADEMAE        int64
               ...   
munResUf       object
munResLat     float64
munResLon     float64
munResAlt     float64
munResArea    float64
Length: 69, dtype: object

In [68]:
# 2) seu código aqui
sinasc.isna().sum()


ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

In [70]:
# 3) seu código aqui
#sinasc_tratado = sinasc.copy()
sinasc = sinasc[['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO',   'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']]
sinasc.dtypes
sinasc.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64

In [72]:
sinasc.shape

(27028, 9)

In [74]:
# 4) seu código aqui
sinasc['APGAR5'].shape

(27028,)

In [76]:
sinasc['APGAR5'].isnull().sum()

103

In [78]:
sinasc['APGAR5'].dropna()


0        10.0
1         9.0
2        10.0
3        10.0
4        10.0
         ... 
27023     9.0
27024     9.0
27025    10.0
27026     9.0
27027     8.0
Name: APGAR5, Length: 26925, dtype: float64

In [80]:
sinasc['APGAR5'].shape

(27028,)

In [82]:
# 5) seu código aqui
sinasc[['ESTCIVMAE', 'CONSULTAS']].isnull().sum()

ESTCIVMAE    317
CONSULTAS      0
dtype: int64

In [84]:
sinasc[['ESTCIVMAE', 'CONSULTAS']]

,ESTCIVMAE,CONSULTAS
0,5.0,4
1,2.0,4
2,9.0,4
3,5.0,3
4,2.0,4
...,...,...
27023,2.0,4
27024,1.0,4
27025,2.0,4
27026,2.0,4


In [86]:
sinasc['ESTCIVMAE'] = sinasc['ESTCIVMAE'].fillna('9')

In [88]:
sinasc[['ESTCIVMAE', 'CONSULTAS']].isnull().sum()

ESTCIVMAE    0
CONSULTAS    0
dtype: int64

In [90]:
sinasc[['ESTCIVMAE', 'CONSULTAS']]

,ESTCIVMAE,CONSULTAS
0,5.0,4
1,2.0,4
2,9.0,4
3,5.0,3
4,2.0,4
...,...,...
27023,2.0,4
27024,1.0,4
27025,2.0,4
27026,2.0,4


In [92]:
# 6) Seu código aqui
sinasc['QTDFILVIVO'] 

0        0.0
1        1.0
2        2.0
3        0.0
4        1.0
        ... 
27023    1.0
27024    0.0
27025    0.0
27026    1.0
27027    1.0
Name: QTDFILVIVO, Length: 27028, dtype: float64

In [94]:
sinasc['QTDFILVIVO'].isnull().sum()

1573

In [96]:
sinasc['QTDFILVIVO'] = sinasc['QTDFILVIVO'].fillna('0')

In [98]:
sinasc['QTDFILVIVO'].isnull().sum()

0

In [100]:
# 7) seu código aqui
# sinasc['LOCNASC', 'IDADEMAE', 'ESCMAE',   'GESTACAO', 'GRAVIDEZ', ]

In [102]:
sinasc.isnull().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         312
QTDFILVIVO       0
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64

In [56]:
sinasc['ESCMAE']

0            8 a 11 anos
1            8 a 11 anos
2            8 a 11 anos
3        12 anos ou mais
4            8 a 11 anos
              ...       
27023    12 anos ou mais
27024        8 a 11 anos
27025        8 a 11 anos
27026        8 a 11 anos
27027        8 a 11 anos
Name: ESCMAE, Length: 27028, dtype: object

In [104]:
#Conforme os dados acima, o melhor seria excluir os dados faltantes
sinasc.dropna(subset=['ESCMAE'], inplace=True)

In [106]:
sinasc.isnull().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE           0
QTDFILVIVO       0
GESTACAO      1197
GRAVIDEZ        78
CONSULTAS        0
APGAR5         101
dtype: int64

In [114]:
sinasc['GESTACAO']

0        37 a 41 semanas
1        37 a 41 semanas
2        37 a 41 semanas
3        37 a 41 semanas
4        37 a 41 semanas
              ...       
27023    32 a 36 semanas
27024    37 a 41 semanas
27025    37 a 41 semanas
27026    32 a 36 semanas
27027    37 a 41 semanas
Name: GESTACAO, Length: 26716, dtype: object

In [116]:
#Conforme os dados acima, o melhor seria excluir os dados faltantes
sinasc.dropna(subset=['GESTACAO'], inplace=True)

In [118]:
sinasc.isnull().sum()

LOCNASC        0
IDADEMAE       0
ESTCIVMAE      0
ESCMAE         0
QTDFILVIVO     0
GESTACAO       0
GRAVIDEZ      62
CONSULTAS      0
APGAR5        85
dtype: int64

In [124]:
sinasc['GRAVIDEZ']

0        Única
1        Única
2        Única
3        Única
4        Única
         ...  
27023    Única
27024    Única
27025    Única
27026    Única
27027    Única
Name: GRAVIDEZ, Length: 25519, dtype: object

In [126]:
#Conforme os dados acima, o melhor seria excluir os dados faltantes
sinasc.dropna(subset=['GRAVIDEZ'], inplace=True)

In [128]:
sinasc.isnull().sum()

LOCNASC        0
IDADEMAE       0
ESTCIVMAE      0
ESCMAE         0
QTDFILVIVO     0
GESTACAO       0
GRAVIDEZ       0
CONSULTAS      0
APGAR5        82
dtype: int64

In [130]:
sinasc['APGAR5']

0        10.0
1         9.0
2        10.0
3        10.0
4        10.0
         ... 
27023     9.0
27024     9.0
27025    10.0
27026     9.0
27027     8.0
Name: APGAR5, Length: 25457, dtype: float64

In [132]:
#Considerando a solicitação do exercício 4, o melhor seria excluir os dados faltantes
sinasc.dropna(subset=['APGAR5'], inplace=True)

In [134]:
sinasc.isnull().sum()

LOCNASC       0
IDADEMAE      0
ESTCIVMAE     0
ESCMAE        0
QTDFILVIVO    0
GESTACAO      0
GRAVIDEZ      0
CONSULTAS     0
APGAR5        0
dtype: int64

In [154]:
# 8) seu código aqui
#Entre 8 e 10 está em uma faixa 'normal'.
#Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'.
#Entre 4 e 5 significa 'asfixia moderada'.
#Entre 0 e 3 significa 'asfixia severa'.
sinasc.loc[sinasc['APGAR5']<4, 'Faixa Asfixia'] = 'asfixia severa'
sinasc.loc[(4<=sinasc['APGAR5']) & (sinasc['APGAR5']<6), 'Faixa Asfixia'] = 'asfixia moderada'
sinasc.loc[(6<=sinasc['APGAR5']) & (sinasc['APGAR5']<8), 'Faixa Asfixia'] = 'asfixia leve'
sinasc.loc[(8<=sinasc['APGAR5']) & (sinasc['APGAR5']<11), 'Faixa Asfixia'] = 'normal'


In [156]:
sinasc[['APGAR5','Faixa Asfixia']]

,APGAR5,Faixa Asfixia
0,10.0,normal
1,9.0,normal
2,10.0,normal
3,10.0,normal
4,10.0,normal
...,...,...
27023,9.0,normal
27024,9.0,normal
27025,10.0,normal
27026,9.0,normal


In [178]:
sinasc['faixa_asfixia'].value_counts()

faixa_asfixia
normal              24946
asfixia leve          302
asfixia severa         68
asfixia moderada       59
Name: count, dtype: int64

In [158]:
sinasc.isnull().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE           0
QTDFILVIVO       0
GESTACAO         0
GRAVIDEZ         0
CONSULTAS        0
APGAR5           0
Faixa Asfixia    0
dtype: int64

In [160]:
# 9) seu código aqui
sinasc.columns

Index(['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO',
       'GRAVIDEZ', 'CONSULTAS', 'APGAR5', 'Faixa Asfixia'],
      dtype='object')

In [164]:
#ano_lista = list(map(lambda value: value[6:10], ano_lista))
sinasc.columns = list(map(lambda value: value.lower(), sinasc.columns))

In [166]:
sinasc.columns

Index(['locnasc', 'idademae', 'estcivmae', 'escmae', 'qtdfilvivo', 'gestacao',
       'gravidez', 'consultas', 'apgar5', 'faixa asfixia'],
      dtype='object')

In [168]:
sinasc.columns = list(map(lambda value: value.replace(' ', '_'), sinasc.columns))

In [170]:
sinasc.columns

Index(['locnasc', 'idademae', 'estcivmae', 'escmae', 'qtdfilvivo', 'gestacao',
       'gravidez', 'consultas', 'apgar5', 'faixa_asfixia'],
      dtype='object')

In [174]:
sinasc.head(1)

,locnasc,idademae,estcivmae,escmae,qtdfilvivo,gestacao,gravidez,consultas,apgar5,faixa_asfixia
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
